# 时间序列预测器

In [1]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

为了使本教程易于理解，我们使用 UCI 空气质量数据集，并尝试预测不同时间步长的 AH 值。 还对数据集执行了一些基本的预处理，因为它需要清理。

## 一个简单的例子

第一步是准备数据。 这里我们以【UCI 空气质量数据集】（https://archive.ics.uci.edu/ml/datasets/Air+Quality）为例。

In [3]:
dataset = tf.keras.utils.get_file(
    fname="AirQualityUCI.csv",
    origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00360/"
    "AirQualityUCI.zip",
    extract=True,
)

In [4]:
dataset

'/home/huangwei/.keras/datasets/AirQualityUCI.csv'

In [5]:
dataset = pd.read_csv(dataset, sep=";")

dataset = dataset[dataset.columns[:-2]]
dataset

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
dataset = dataset.dropna()
dataset = dataset.replace(",", ".", regex=True)
dataset

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004,19.00.00,2,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,04/04/2005,10.00.00,3.1,1314.0,-200.0,13.5,1101.0,472.0,539.0,190.0,1374.0,1729.0,21.9,29.3,0.7568
9353,04/04/2005,11.00.00,2.4,1163.0,-200.0,11.4,1027.0,353.0,604.0,179.0,1264.0,1269.0,24.3,23.7,0.7119
9354,04/04/2005,12.00.00,2.4,1142.0,-200.0,12.4,1063.0,293.0,603.0,175.0,1241.0,1092.0,26.9,18.3,0.6406
9355,04/04/2005,13.00.00,2.1,1003.0,-200.0,9.5,961.0,235.0,702.0,156.0,1041.0,770.0,28.3,13.5,0.5139


In [7]:
val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

In [8]:
data_x = data_train[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

In [9]:
data_x_val = validation_data[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

In [10]:
# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

In [11]:
data_y = data_train["AH"].astype("float64")

In [12]:
data_y_val = validation_data["AH"].astype("float64")

print(data_x.shape)  # (6549, 12)
print(data_y.shape)  # (6549,)

(6549, 12)
(6549,)


In [13]:
data_x

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH
0,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9
1,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7
2,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0
3,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0
4,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6
...,...,...,...,...,...,...,...,...,...,...,...,...
6544,1.7,1111.0,-200.0,7.6,884.0,310.0,751.0,81.0,1280.0,1025.0,13.1,66.2
6545,2.3,1206.0,-200.0,9.8,969.0,363.0,701.0,95.0,1365.0,1178.0,14.3,63.3
6546,2.1,1202.0,-200.0,9.5,958.0,302.0,732.0,92.0,1330.0,1181.0,16.0,58.1
6547,2.7,1261.0,-200.0,11.2,1022.0,424.0,635.0,113.0,1407.0,1250.0,17.3,53.9


第二步是运行 TimeSeriesForecaster。 作为一个快速演示，我们将 epochs 设置为 10。您也可以为自适应数量的 epochs 保留未指定的 epochs。

In [16]:
predict_from = 1
predict_until = 10
lookback = 3
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=10,
    objective="val_loss",
)

INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


In [18]:
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
     epochs=10,
)

Trial 10 Complete [00h 00m 50s]
val_loss: 2.1160502433776855

Best val_loss So Far: 2.1160502433776855
Total elapsed time: 00h 08m 04s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
205/205 [==============================] - 9s 26ms/step - loss: 544.4609 - mean_squared_error: 544.4609 - val_loss: 733.0435 - val_mean_squared_error: 733.0435
Epoch 2/10
205/205 [==============================] - 4s 18ms/step - loss: 38.4624 - mean_squared_error: 38.4624 - val_loss: 397.4051 - val_mean_squared_error: 397.4051
Epoch 3/10
205/205 [==============================] - 3s 16ms/step - loss: 7.4484 - mean_squared_error: 7.4484 - val_loss: 310.8840 - val_mean_squared_error: 310.8840
Epoch 4/10
205/205 [==============================] - 4s 17ms/step - loss: 1.1156 - mean_squared_error: 1.1156 - val_loss: 324.0226 - val_mean_squared_error: 324.0226
Epoch 5/10
205/205 [==============================] - 4s 18ms/step - loss: 0.8045 - mean_squared_error: 0.8045 - val_loss: 326.0412 - val_mean_squared_error: 326.0412
Epoch 6/10
205/205 [==============================] - 4s 18ms/step - loss: 0.6095 - mean_squared_error: 0.6095 - val_loss: 327.0600 - val_mean_squared_error: 3

INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


In [20]:
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)

293/293 [==============================] - 2s 7ms/step
(10, 1)


In [21]:
predictions

array([[1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169],
       [1.0209169]], dtype=float32)

In [22]:
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))

88/88 [==============================] - 2s 9ms/step - loss: 327.6228 - mean_squared_error: 327.6228
[327.6228332519531, 327.6228332519531]


In [23]:
model = clf.export_model()

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 12)]           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 3, 24)             2400      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 24)                3552      
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 25        
Total params: 5,977
Trainable params: 5,977
Non-trainable params: 0
_________________________________________________________________
